This notebook trains a logistic regression model, finds the optimal value of C, and reports F1 and log loss scores.

In [1]:
import numpy as np
import pandas as pd
import zipfile
from sklearn import metrics
from sklearn.linear_model import LogisticRegression

In [ ]:
# Unzip data files into the "csv" subdirectory 
# (unless you have already done this since running the Data Set Up notebook)

# **IMPORTANT**  This will overwrite existing files in the "csv" folder in your local repo
# with the most recent data files from the data.zip file

# Unzip 80% training data
unzip_training_data = zipfile.ZipFile("data_subset.zip", "r")
unzip_training_data.extractall()
unzip_training_data.close()

# Unzip development and training data
unzip_test_data = zipfile.ZipFile("testing.zip", "r")
unzip_test_data.extractall()
unzip_test_data.close()

# Unzip full set of training data for creating predictions to submit to Kaggle
unzip_all_data = zipfile.ZipFile("data.zip", "r")
unzip_all_data.extractall()
unzip_all_data.close()

In [2]:
# Load these csv files into numpy arrays for testing on development data
train_data = np.loadtxt('csv/train_data.csv', delimiter=",")
train_labels = np.loadtxt('csv/train_labels.csv', dtype=str, delimiter=",")
dev_data = np.loadtxt('csv/dev_data.csv', delimiter=",")
dev_labels = np.loadtxt('csv/dev_labels.csv', dtype=str, delimiter=",")

In [ ]:
# Load these csv files into numpy arrays for creating predictions to submit to Kaggle
train_data_all = np.loadtxt('csv/train_data_all.csv', delimiter=",")
train_labels_all = np.loadtxt('csv/train_labels_all.csv', dtype=str, delimiter=",")
test_data_all = np.loadtxt('csv/test_data_all.csv', delimiter=",")

In [5]:
# print shapes to compare before and after csv conversion
print("train_data shape is", train_data.shape)
print("train_labels shape is", train_labels.shape)
print("dev_data shape is", dev_data.shape)
print("dev_labels shape is", dev_labels.shape)

train_data shape is (702439, 58)
train_labels shape is (702439,)
dev_data shape is (175610, 58)
dev_labels shape is (175610,)


In [ ]:
print("train_data_all shape is", train_data_all.shape)
print("train_labels_all shape is", train_labels_all.shape)
print("test_data_all shape is", test_data_all.shape)

In [7]:
# Set up functions for training logistic regression model and finding optimal value of C

def TrainLR(data, labels, test_data, C_value=1.0):
    """This function takes in training data and labels, testing data,
    and can accept different values of C (the learning rate).
    It trains a logistic regression model and returns the model and predicted probabilities.
    """
    LR = LogisticRegression(C=C_value)
    LR.fit(data, labels)
    pp = LR.predict_proba(test_data)
    return LR, pp

def find_C(data, labels, dev_data, dev_labels, C_values):
    """Find optimal value of C in a logistic regression model.  
    
    Note that this cannot be used on test data from Kaggle 
    because we do not have labels for that data.  This function is intended to only be used
    in the development stage with the development data.
    """
    for C in C_values:      
        LR, pp = TrainLR(data, labels, dev_data, C)
        predictions = LR.predict(dev_data)
        f1 = metrics.f1_score(dev_labels, predictions, average = "weighted")
        logloss = metrics.log_loss(dev_labels, pp)
        
        # Print F1 score and log loss for each value of k
        print("For C =", C, "the F1 score is", round(f1, 4), "and the Log Loss score is", round(logloss, 4))
    print("\n")

In [ ]:
# IF there are additional changes to make to the data for this model
# that would be easier to do in pandas, uncomment and run this code. 
# This model works the same whether the data is in numpy or pandas, so presumably so do other models

#train_data = pd.DataFrame(train_data)
#train_labels = pd.DataFrame(train_labels)
#dev_data = pd.DataFrame(dev_data)
#dev_labels = pd.DataFrame(dev_labels)
#train_data_all = pd.DataFrame(train_data_all)
#train_labels_all = pd.DataFrame(train_labels_all)
#test_data_all = pd.DataFrame(test_data_all)

In [ ]:
# Find the optimal value of C using the 80% training data and the development data
C_values = [0.0001, 0.001, 0.01, 0.1, 0.5, 1.0, 2.0, 10.0, 100.00, 1000.0]
find_C(train_data, train_labels, dev_data, dev_labels, C_values)

In [4]:
# Train model with a single value of C with 80% training data and development data
C_value = 0.5
LR, pp = TrainLR(train_data, train_labels, dev_data, C_value)
logloss = metrics.log_loss(dev_labels, pp)
print(logloss)

2.54342653545


In [ ]:
# Before submitting to Kaggle, run the model on the full set of training data and test data
# using the optimal value for the model
C_value = use optimal C value
LR, pp = TrainLR(train_data_all, train_labels_all, test_data_all, C_value)

In [ ]:
# Set up predictions for submission to Kaggle
headers = ["ARSON","ASSAULT","BAD CHECKS","BRIBERY","BURGLARY","DISORDERLY CONDUCT","DRIVING UNDER THE INFLUENCE",
           "DRUG/NARCOTIC","DRUNKENNESS","EMBEZZLEMENT","EXTORTION","FAMILY OFFENSES","FORGERY/COUNTERFEITING",
           "FRAUD","GAMBLING","KIDNAPPING","LARCENY/THEFT","LIQUOR LAWS","LOITERING","MISSING PERSON","NON-CRIMINAL",
           "OTHER OFFENSES","PORNOGRAPHY/OBSCENE MAT","PROSTITUTION","RECOVERED VEHICLE","ROBBERY","RUNAWAY",
           "SECONDARY CODES","SEX OFFENSES FORCIBLE","SEX OFFENSES NON FORCIBLE","STOLEN PROPERTY","SUICIDE",
           "SUSPICIOUS OCC","TREA","TRESPASS","VANDALISM","VEHICLE THEFT","WARRANTS","WEAPON LAWS"]
data = pd.DataFrame(data=pp, 
                    index=[x for x in range(len(test_data_all))], 
                    columns=headers)
data.columns.name ="Id"
print(data)Create zipped csv file for Kaggle
#### Update the filename first in all lines of the following code
Add something unique after our names to avoid overwriting other submission files

Create zipped csv file for Kaggle
#### Update the filename first in all lines of the following code
Add something unique after our names to avoid overwriting other submission files

In [ ]:
data.to_csv('Williams_Gascoigne_Vignola_.csv', index_label = "Id")

In [ ]:
zip_probs = zipfile.ZipFile("Williams_Gascoigne_Vignola_.zip", "w")
zip_probs.write("Williams_Gascoigne_Vignola_.csv", compress_type=zipfile.ZIP_DEFLATED)
zip_probs.close()

### Results from previous datasets and/or model parameters